In [160]:
import pandas as pd
import numpy as np
from pyomo.environ import * 
from pyomo.opt import SolverFactory
import folium
import csv

In [161]:
clients_df = pd.read_csv("Proyecto_A_Caso3/clients.csv")
depots_df = pd.read_csv("Proyecto_A_Caso3/depots.csv")
vehicles_df = pd.read_csv("Proyecto_A_Caso3/vehicles.csv")



In [162]:
print(clients_df.head())
print(depots_df.head())
print(vehicles_df.head())


   ClientID  locationID  Demand  Longitude  Latitude
0         1          12      12 -74.150806  4.679769
1         2          13      12 -74.017996  4.733831
2         3          14      12 -74.143655  4.620928
3         4          15      12 -74.034583  4.733074
4         5          16      12 -74.055887  4.820900
   DepotID  LocationID  Longitude  Latitude  Capacity
0        1           1 -74.081242  4.750212        11
1        2           2 -74.109934  4.536383        90
2        3           3 -74.038548  4.792926       130
3        4           4 -74.067069  4.721678       145
4        5           5 -74.138263  4.607707       260
   VehicleType  Capacity  Range
0            1       132    146
1            2       136    196
2            3       115    143
3            4       158    174
4            5       109    167


In [163]:
C=list(depots_df["DepotID"])
CAP=dict(zip(depots_df["DepotID"],depots_df["Capacity"]))


In [164]:
print("Conjunto C:", C)
print("Capacidades CAP_i:", CAP)


Conjunto C: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Capacidades CAP_i: {1: 11, 2: 90, 3: 130, 4: 145, 5: 260, 6: 180, 7: 720, 8: 55, 9: 70, 10: 75, 11: 90, 12: 270}


In [165]:
K=list(clients_df["ClientID"])
DEM=dict(zip(clients_df["ClientID"],clients_df["Demand"]))  # Esta es la demanda que solicita ese cliente

In [166]:
print("Conjunto K: clientes", K)
print("Capacidades Demanda:", DEM)

Conjunto K: clientes [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90]
Capacidades Demanda: {1: 12, 2: 12, 3: 12, 4: 12, 5: 12, 6: 12, 7: 12, 8: 12, 9: 12, 10: 12, 11: 12, 12: 12, 13: 8, 14: 12, 15: 12, 16: 12, 17: 12, 18: 12, 19: 12, 20: 12, 21: 12, 22: 12, 23: 12, 24: 12, 25: 12, 26: 12, 27: 12, 28: 12, 29: 12, 30: 12, 31: 11, 32: 12, 33: 12, 34: 12, 35: 12, 36: 12, 37: 12, 38: 12, 39: 12, 40: 12, 41: 12, 42: 12, 43: 12, 44: 12, 45: 12, 46: 12, 47: 12, 48: 12, 49: 12, 50: 12, 51: 12, 52: 12, 53: 12, 54: 12, 55: 12, 56: 12, 57: 9, 58: 12, 59: 12, 60: 12, 61: 12, 62: 12, 63: 12, 64: 12, 65: 12, 66: 12, 67: 12, 68: 12, 69: 12, 70: 12, 71: 12, 72: 12, 73: 12, 74: 12, 75: 12, 76: 12, 77: 12, 

Conjunto para la distancia

In [167]:
def dist_haversiana(lon1,lat1,lon2,lat2):
  R=6371.0 #radio tierra
  lon1,lat1,lon2,lat2=map(np.radians,[lon1,lat1,lon2,lat2])
  dlon=lon2-lon1
  dlat=lat2-lat1
  a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
  c = 2 * np.arcsin(np.sqrt(a))
  return R * c



In [168]:
DIST={}
for i,depot in depots_df.iterrows():
  for j,client in clients_df.iterrows():
    di=depot["DepotID"]
    ci=client["ClientID"]
    distance=dist_haversiana(depot["Longitude"],depot["Latitude"],
                             client["Longitude"],client["Latitude"]
                             )
    DIST[(di,ci)]=round(distance,2)


In [169]:
for key, val in list(DIST.items())[:5]:
  print("La distancia entre el centro de distribucion", key[0], "y el cliente", key[1], "es de", val)

La distancia entre el centro de distribucion 1.0 y el cliente 1.0 es de 10.99
La distancia entre el centro de distribucion 1.0 y el cliente 2.0 es de 7.24
La distancia entre el centro de distribucion 1.0 y el cliente 3.0 es de 15.95
La distancia entre el centro de distribucion 1.0 y el cliente 4.0 es de 5.51
La distancia entre el centro de distribucion 1.0 y el cliente 5.0 es de 8.35


Procesamiento de vehiculos

In [170]:
vehicles_df

,VehicleType,Capacity,Range
0,1,132,146
1,2,136,196
2,3,115,143
3,4,158,174
4,5,109,167
5,6,109,137
6,7,126,93
7,8,79,134
8,9,96,160
9,10,99,108


In [171]:
V=list(vehicles_df.index)
Q = dict(zip(V, vehicles_df["Capacity"]))


R = dict(zip(V, vehicles_df["Range"]))



In [172]:
print("Vehículos:", V)
print("Capacidades:", Q)
print("Rangos:", R)


Vehículos: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Capacidades: {0: 132, 1: 136, 2: 115, 3: 158, 4: 109, 5: 109, 6: 126, 7: 79, 8: 96, 9: 99, 10: 125, 11: 98, 12: 86, 13: 74, 14: 85, 15: 22, 16: 25, 17: 30, 18: 27, 19: 19, 20: 29, 21: 25, 22: 18, 23: 23, 24: 27, 25: 27, 26: 22, 27: 31, 28: 21, 29: 27}
Rangos: {0: 146, 1: 196, 2: 143, 3: 174, 4: 167, 5: 137, 6: 93, 7: 134, 8: 160, 9: 108, 10: 955, 11: 716, 12: 1023, 13: 1076, 14: 942, 15: 21, 16: 12, 17: 12, 18: 10, 19: 16, 20: 16, 21: 15, 22: 13, 23: 19, 24: 10, 25: 14, 26: 17, 27: 18, 28: 15, 29: 18}


In [173]:
for v in V:
    print(f"vehiculo {v}: Tipo={vehicles_df['VehicleType'][v]}, Capacidad={Q[v]}, Rango={R[v]}")

vehiculo 0: Tipo=1, Capacidad=132, Rango=146
vehiculo 1: Tipo=2, Capacidad=136, Rango=196
vehiculo 2: Tipo=3, Capacidad=115, Rango=143
vehiculo 3: Tipo=4, Capacidad=158, Rango=174
vehiculo 4: Tipo=5, Capacidad=109, Rango=167
vehiculo 5: Tipo=6, Capacidad=109, Rango=137
vehiculo 6: Tipo=7, Capacidad=126, Rango=93
vehiculo 7: Tipo=8, Capacidad=79, Rango=134
vehiculo 8: Tipo=9, Capacidad=96, Rango=160
vehiculo 9: Tipo=10, Capacidad=99, Rango=108
vehiculo 10: Tipo=11, Capacidad=125, Rango=955
vehiculo 11: Tipo=12, Capacidad=98, Rango=716
vehiculo 12: Tipo=13, Capacidad=86, Rango=1023
vehiculo 13: Tipo=14, Capacidad=74, Rango=1076
vehiculo 14: Tipo=15, Capacidad=85, Rango=942
vehiculo 15: Tipo=16, Capacidad=22, Rango=21
vehiculo 16: Tipo=17, Capacidad=25, Rango=12
vehiculo 17: Tipo=18, Capacidad=30, Rango=12
vehiculo 18: Tipo=19, Capacidad=27, Rango=10
vehiculo 19: Tipo=20, Capacidad=19, Rango=16
vehiculo 20: Tipo=21, Capacidad=29, Rango=16
vehiculo 21: Tipo=22, Capacidad=25, Rango=15
vehic

# Creacion del modelo

In [174]:
model=ConcreteModel()

#Definicion de conjuntos
model.C= Set(initialize=C) # Centros de distribucion
model.K = Set(initialize=K) # Clientes
model.V = Set(initialize=V) # Vehículos
model.N = model.C | model.K    # Todos los nodos CDs + Clientes el | hace la unicoon de los conjuntos


In [175]:
# Parámetros
model.CAP = Param(model.C, initialize=CAP)
model.DEM = Param(model.K, initialize=DEM)
model.Q = Param(model.V, initialize=Q)
model.R = Param(model.V, initialize=R)

# Distancias entre centros y clientes
model.DIST = Param(model.C, model.K, initialize=DIST, within=NonNegativeReals)

Pf = 123.12  # COP/km (combustible)
Ft = 823     # COP/km (flete)
Cm = 700     # COP/km (mantenimiento)


### Variables de decision

In [176]:
model.x =Var(model.C,model.K,model.V,domain=Binary) #X{i,j,l}
model.y= Var(model.C,model.V,domain=Binary) #y{i,l}
model.I=Var(model.C,domain=NonNegativeReals) # Inventario asignado a los centros de distribucion
model.u= Var(model.K,model.V,domain=NonNegativeReals) # Carga entregada en cada punto de distribucion



### Funcion objetivo

In [177]:
def objetivo(model):
  return sum(
    (Pf+Ft+Cm)* model.DIST[i,j]*model.x[i,j,l]
    for i in model.C for j in model.K for l in model.V
  )

model.OBJ=Objective(rule=objetivo,sense=minimize)


####  Restricciones

Capacidad de los Centros de Distribución

In [178]:
def restriccionCapacidad_CentroDistribucion(model,i):
  return model.I[i]<=model.CAP[i]

model.RestriccionCapacidad_CentroDistribucion=Constraint(model.C, rule=restriccionCapacidad_CentroDistribucion)

Satisfacción de la demanda de cada cliente

In [179]:
def restriccion_demanda(model, j):
    return sum(model.u[j, l] for l in model.V) == model.DEM[j]

model.RestriccionDemanda = Constraint(model.K, rule=restriccion_demanda)

Capacidad del vehículo

In [180]:
def restriccion_capacidad_vehiculo(model, l):
    return sum(model.u[j, l] for j in model.K) <= sum(model.Q[l] * model.y[i, l] for i in model.C)

model.RestriccionCapacidadVehiculo = Constraint(model.V, rule=restriccion_capacidad_vehiculo)


Asignar cada vehículo a un único centro de distribución

In [181]:
def restriccion_asignacion_vehiculos(model, l):
    return sum(model.y[i, l] for i in model.C) == 1

model.RestriccionAsignacionVehiculo = Constraint(model.V, rule=restriccion_asignacion_vehiculos)


Rango util del vehiculo

In [182]:
def restriccion_rango(model, l):
    return sum(model.DIST[i, j] * model.x[i, j, l] for i in model.C for j in model.K) <= model.R[l]

model.RestriccionRango = Constraint(model.V, rule=restriccion_rango)


In [183]:
def link_xy_rule(m, i, j, l):
    return m.x[i, j, l] <= m.y[i, l]
model.link_xy = Constraint(model.C, model.K, model.V, rule=link_xy_rule)

In [184]:
def link_ux_rule(m, j, l):
    return m.u[j, l] <= m.Q[l] * sum(m.x[i, j, l] for i in m.C)
model.link_ux = Constraint(model.K, model.V, rule=link_ux_rule)


In [185]:
def visita_rule(m, j):
    return sum(m.x[i, j, l] for i in m.C for l in m.V) == 1
model.visita = Constraint(model.K, rule=visita_rule)

In [186]:
def origen_rule(m, l):
    return sum(m.y[i, l] for i in m.C) == 1
model.un_origen = Constraint(model.V, rule=origen_rule)

resolver el modelo

In [187]:
solver = SolverFactory('glpk')
# Resolver
result = solver.solve(model, tee=True)

# Mostrar el estado y el valor de la función objetivo
print("Estado:", result.solver.status)
print("Óptimo:", value(model.OBJ))


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\jorgi\AppData\Local\Temp\tmpdstu98bh.glpk.raw --wglp C:\Users\jorgi\AppData\Local\Temp\tmpkrgisq0g.glpk.glp
 --cpxlp C:\Users\jorgi\AppData\Local\Temp\tmpgpk08ucg.pyomo.lp
Reading problem data from 'C:\Users\jorgi\AppData\Local\Temp\tmpgpk08ucg.pyomo.lp'...
C:\Users\jorgi\AppData\Local\Temp\tmpgpk08ucg.pyomo.lp:345310: warning: lower bound of variable 'x2' redefined
C:\Users\jorgi\AppData\Local\Temp\tmpgpk08ucg.pyomo.lp:345310: warning: upper bound of variable 'x2' redefined
35412 rows, 35472 columns, 171192 non-zeros
32760 integer variables, all of which are binary
378070 lines were read
Writing problem data to 'C:\Users\jorgi\AppData\Local\Temp\tmpkrgisq0g.glpk.glp'...
312603 lines were written
GLPK Integer Optimizer 5.0
35412 rows, 35472 columns, 171192 non-zeros
32760 integer variables, all of which are binary
Preprocessing...
28037 constraint coefficient(s) were reduced
31037 rows, 31097 c

In [188]:
for l in model.V:
    # Crear un registro de la ruta
    ruta = []
    
    # Buscamos el centro de distribución desde el que parte el vehículo
    for i in model.C:
        if model.y[i, l].value == 1:
            ruta.append(f"CD {i}")
            break  # Solo un centro de distribución

    # Luego, buscamos a qué clientes va el vehículo
    cliente_anterior = None
    for i in model.C:
        for j in model.K:
            if model.x[i, j, l].value == 1: 
                if cliente_anterior:
                    ruta.append(f"Cliente {cliente_anterior} a Cliente {j}")
                else:
                    ruta.append(f"CD {i} a Cliente {j}")
                cliente_anterior = j 
               
    print(f"Vehículo {l} está en este punto y sigue esta ruta: {' -> '.join(ruta)}")


Vehículo 0 está en este punto y sigue esta ruta: CD 4 -> CD 4 a Cliente 2 -> Cliente 2 a Cliente 7 -> Cliente 7 a Cliente 12 -> Cliente 12 a Cliente 20 -> Cliente 20 a Cliente 63 -> Cliente 63 a Cliente 66 -> Cliente 66 a Cliente 67 -> Cliente 67 a Cliente 72 -> Cliente 72 a Cliente 81
Vehículo 1 está en este punto y sigue esta ruta: CD 5 -> CD 5 a Cliente 3 -> Cliente 3 a Cliente 16 -> Cliente 16 a Cliente 19 -> Cliente 19 a Cliente 33 -> Cliente 33 a Cliente 42 -> Cliente 42 a Cliente 46 -> Cliente 46 a Cliente 51 -> Cliente 51 a Cliente 60 -> Cliente 60 a Cliente 64 -> Cliente 64 a Cliente 65 -> Cliente 65 a Cliente 76
Vehículo 2 está en este punto y sigue esta ruta: CD 6 -> CD 6 a Cliente 13 -> Cliente 13 a Cliente 32
Vehículo 3 está en este punto y sigue esta ruta: CD 7 -> CD 7 a Cliente 73
Vehículo 4 está en este punto y sigue esta ruta: CD 3 -> CD 3 a Cliente 31 -> Cliente 31 a Cliente 77
Vehículo 5 está en este punto y sigue esta ruta: CD 6 -> CD 6 a Cliente 1 -> Cliente 1 a Cl

In [189]:
for i in model.C:
    for l in model.V:
        if model.y[i, l].value == 1:
            print(f"Vehículo {l} parte desde CD {i}")


Vehículo 22 parte desde CD 1
Vehículo 23 parte desde CD 2
Vehículo 4 parte desde CD 3
Vehículo 15 parte desde CD 3
Vehículo 18 parte desde CD 3
Vehículo 0 parte desde CD 4
Vehículo 7 parte desde CD 4
Vehículo 19 parte desde CD 4
Vehículo 1 parte desde CD 5
Vehículo 25 parte desde CD 5
Vehículo 27 parte desde CD 5
Vehículo 2 parte desde CD 6
Vehículo 5 parte desde CD 6
Vehículo 17 parte desde CD 6
Vehículo 3 parte desde CD 7
Vehículo 6 parte desde CD 7
Vehículo 20 parte desde CD 8
Vehículo 24 parte desde CD 8
Vehículo 26 parte desde CD 8
Vehículo 28 parte desde CD 8
Vehículo 14 parte desde CD 9
Vehículo 8 parte desde CD 10
Vehículo 16 parte desde CD 10
Vehículo 12 parte desde CD 11
Vehículo 29 parte desde CD 11
Vehículo 9 parte desde CD 12
Vehículo 10 parte desde CD 12
Vehículo 11 parte desde CD 12
Vehículo 13 parte desde CD 12
Vehículo 21 parte desde CD 12


In [190]:
mapa = folium.Map(location=[4.60971, -74.08175], zoom_start=12)

for _, depot in depots_df.iterrows():
    folium.Marker(
        location=[depot['Latitude'], depot['Longitude']], 
        popup=f"CD {depot['DepotID']}", 
        icon=folium.Icon(color='blue')
    ).add_to(mapa)

for _, client in clients_df.iterrows():
    folium.Marker(
        location=[client['Latitude'], client['Longitude']], 
        popup=f"Cliente {client['ClientID']}",
        icon=folium.Icon(color='green')
    ).add_to(mapa)

for l in model.V:
    ruta_coordenadas = []

    for i in model.C:
        if model.y[i, l].value == 1:
            cd_lat = depots_df[depots_df['DepotID'] == i]['Latitude'].values[0]
            cd_lon = depots_df[depots_df['DepotID'] == i]['Longitude'].values[0]
            ruta_coordenadas.append([cd_lat, cd_lon])
            break

    cliente_anterior = None
    for i in model.C:
        for j in model.K:
            if model.x[i, j, l].value == 1:
                client_lat = clients_df[clients_df['ClientID'] == j]['Latitude'].values[0]
                client_lon = clients_df[clients_df['ClientID'] == j]['Longitude'].values[0]
                ruta_coordenadas.append([client_lat, client_lon])
                
                cliente_anterior = j

    folium.PolyLine(ruta_coordenadas, color='red', weight=2.5, opacity=1).add_to(mapa)

mapa
mapa.save("./caso3.html")

In [191]:
# Definimos la velocidad promedio en Bogotá para un furgón
velocidad_promedio = 40  # km/h

# Calcular el tiempo total basado en la distancia recorrida
def calcular_tiempo_total(distancia_km):
    tiempo_horas = distancia_km / velocidad_promedio 
    return tiempo_horas * 60 


def guardar_verificacion(model, archivo='verificacion_caso.csv'):
    with open(archivo, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['VehicleId', 'DepotId', 'InitialLoad', 'RouteSequence', 'ClientsServed', 
                         'DemandsSatisfied', 'TotalDistance', 'TotalTime', 'FuelCost'])

        # Recorremos los vehículos
        for l in model.V:
            vehicle_id = f"{l+1:03d}"
            
            depot_id = None
            for i in model.C:
                if model.y[i, l].value == 1:
                    depot_id = i
                    break
            
            initial_load = model.I[depot_id].value if depot_id is not None else 0
            
            route_sequence = []
            cliente_anterior = None
            total_distance = 0  # Distancia total
            for i in model.C:
                for j in model.K:
                    if model.x[i, j, l].value == 1:
                        if cliente_anterior:
                            route_sequence.append(f"C{cliente_anterior} - C{j}")
                        else:
                            route_sequence.append(f"CD {i} - C{j}")
                        cliente_anterior = j
                        total_distance += model.DIST[i, j]
            
            # Clientes atendidos
            clients_served = len(route_sequence)
            
            demands_satisfied = []
            for j in model.K:
                if cliente_anterior == j:
                    demands_satisfied.append(str(model.DEM[j])) 

            total_time = calcular_tiempo_total(total_distance)
            
            fuel_cost = total_distance * (Pf + Ft + Cm)
            
            writer.writerow([
                vehicle_id,
                depot_id,
                initial_load,
                '-'.join(route_sequence),
                clients_served,
                '-'.join(demands_satisfied),
                total_distance,
                total_time,
                fuel_cost
            ])

guardar_verificacion(model, archivo='verificacion_caso1.csv')


In [1]:
import pandas as pd

df = pd.read_csv("Archivos_Verificacion/verificacion_caso3.csv")
df_usados = df[df["ClientsServed"] > 0]

dist_prom = df_usados["TotalDistance"].mean()
tiempo_prom = df_usados["TotalTime"].mean()
carga_prom = df_usados["DemandsSatisfied"].astype(str).str.replace("-", "").str.replace(" ", "").str.split(",").explode().astype(float).groupby(level=0).sum().mean()

print(f"Distancia promedio: {dist_prom:.2f} km")
print(f"Tiempo promedio: {tiempo_prom:.2f} min")
print(f"Carga promedio: {carga_prom:.2f} unidades")

Distancia promedio: 11.56 km
Tiempo promedio: 17.34 min
Carga promedio: 11.92 unidades
